<a href="https://colab.research.google.com/github/KORALLLL/MTUCI_EMNIST/blob/main/bagging.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/KORALLLL/MTUCI_EMNIST.git

Cloning into 'MTUCI_EMNIST'...
remote: Enumerating objects: 36671, done.
remote: Counting objects: 100% (13031/13031), done.
remote: Compressing objects: 100% (12853/12853), done.
remote: Total 36671 (delta 298), reused 12831 (delta 164), pack-reused 23640
Receiving objects: 100% (36671/36671), 140.59 MiB | 13.77 MiB/s, done.
Resolving deltas: 100% (583/583), done.
Filtering content: 100% (6/6), 192.63 MiB | 59.18 MiB/s, done.


# prepairing to ansamblinh

In [1]:
import torch
import pickle
import numpy as np
from MTUCI_EMNIST.models.alex import LeNetDropOut as model_alex
from MTUCI_EMNIST.models.artem import Net5 as model_artem
from MTUCI_EMNIST.models.gleb import LeNet as model_gleb
from MTUCI_EMNIST.models.kirill import Model5 as model_kirill
from MTUCI_EMNIST.models.nastya import Lenet4 as model_nastya

In [2]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

file = open('MTUCI_EMNIST/dataset.pkl', 'rb')
test_dataset = pickle.load(file)
file.close()

test_data = test_dataset['data'].numpy()
test_data = np.flip(test_data, axis = 3)
test_data = np.rot90(test_data, k=1, axes=(2,3))
test_data = 1 - test_data
test_data = torch.from_numpy(test_data).float().to(device)
test_labels = test_dataset['targets'].to(device)

In [ ]:
# # путь к исходному файлу
# input_file_path = 'weights\Artem_ModelParameters0.8530000448226929.pth'

# # пути для двух выходных файлов
# output_file_path_part1 = 'your_model_weights_part1.pth'
# output_file_path_part2 = 'your_model_weights_part2.pth'

# # размер каждой части (в байтах)
# chunk_size = 100000000  # н100 МБ

# with open(input_file_path, 'rb') as input_file:
#     #  данные из исходного файла
#     data = input_file.read()

#     #  данные на две части
#     data_part1 = data[:chunk_size]
#     data_part2 = data[chunk_size:]

#     #  данные в выходные файлы
#     with open(output_file_path_part1, 'wb') as output_file_part1:
#         output_file_part1.write(data_part1)

#     with open(output_file_path_part2, 'wb') as output_file_part2:
#         output_file_part2.write(data_part2)


In [3]:
def combine_files(input_file_path_part1, input_file_path_part2):
    with open(input_file_path_part1, 'rb') as input_file_part1, \
         open(input_file_path_part2, 'rb') as input_file_part2:

        #  данные из первой и второй частей
        data_part1 = input_file_part1.read()
        data_part2 = input_file_part2.read()

        # Объединяем данные
        combined_data = data_part1 + data_part2

    return combined_data

# пути для ваших файлов
input_file_path_part1 = '/content/MTUCI_EMNIST/weights/your_model_weights_part1.pth'
input_file_path_part2 = '/content/MTUCI_EMNIST/weights/your_model_weights_part2.pth'

# функцию для объединения данных в одну переменную
combined_data = combine_files(input_file_path_part1, input_file_path_part2)


# Artem net

In [31]:
from io import BytesIO
import gc

net1 = model_artem().to(device)
net1.load_state_dict(torch.load(BytesIO(combined_data), map_location=device))
net1.eval()
with torch.no_grad():
  test_preds = torch.sigmoid(net1.forward(test_data)).float().cpu().data

In [32]:
print(((test_preds.argmax(dim=1) == test_labels).float().mean().cpu().data))

tensor(0.8534)


In [33]:
result = test_preds*0.8534

In [34]:
print(((result.argmax(dim=1) == test_labels).float().mean().cpu().data))

tensor(0.8534)


In [35]:
del net1
del test_preds
torch.cuda.empty_cache()
gc.collect()

9

# Sasha net

In [36]:
net = model_alex(0.1).to(device)
net.load_state_dict(torch.load('/content/MTUCI_EMNIST/weights/Sasha_ModelParameters0.8540592193603516.pth', map_location=device))
net.eval()
with torch.no_grad():
  test_preds = torch.sigmoid(net.forward(test_data)).float().cpu().data
print(((test_preds.argmax(dim=1) == test_labels).float().mean().cpu().data))

tensor(0.8539)


In [37]:
result += (test_preds*0.8539)

In [38]:
print(((result.argmax(dim=1) == test_labels).float().mean().cpu().data))

tensor(0.8665)


In [39]:
del net
del test_preds
torch.cuda.empty_cache()
gc.collect()

10

# Nastya net

In [40]:
net = model_nastya().to(device)
net.load_state_dict(torch.load('/content/MTUCI_EMNIST/weights/Nastya_ModelParameters0.8714902997016907.pth', map_location=device))
net.eval()
with torch.no_grad():
  test_preds = torch.sigmoid(net.forward(test_data)).float().cpu().data
print(((test_preds.argmax(dim=1) == test_labels).float().mean().cpu().data))

tensor(0.8718)


In [41]:
result += (test_preds*0.8718)

In [42]:
print(((result.argmax(dim=1) == test_labels).float().mean().cpu().data))

tensor(0.8765)


In [43]:
del net
del test_preds
torch.cuda.empty_cache()
gc.collect()

9

# Gleb net

In [44]:
net = model_gleb().to(device)
net.load_state_dict(torch.load('/content/MTUCI_EMNIST/weights/Gleb_ModelParameters0.8664226531982422.pth', map_location=device))
net.eval()
with torch.no_grad():
  test_preds = torch.sigmoid(net.forward(test_data)).float().cpu().data
print(((test_preds.argmax(dim=1) == test_labels).float().mean().cpu().data))

tensor(0.8667)


In [45]:
result += (test_preds*0.8667)

In [46]:
print(((result.argmax(dim=1) == test_labels).float().mean().cpu().data))

tensor(0.8793)


In [47]:
del net
del test_preds
torch.cuda.empty_cache()
gc.collect()

9

# Kirill Net

In [48]:
net = model_kirill().to(device)
net.load_state_dict(torch.load('/content/MTUCI_EMNIST/weights/Kirill_ModelParameters0.8665027618408203(1).pth', map_location=device))
net.eval()
with torch.no_grad():
  test_preds = torch.sigmoid(net.forward(test_data)).float().cpu().data
print(((test_preds.argmax(dim=1) == test_labels).float().mean().cpu().data))

tensor(0.8671)


In [56]:
result += (test_preds*0.8671)

# scoring

In [57]:
print(((result.argmax(dim=1) == test_labels).float().mean().cpu().data))

tensor(0.8815)
